In [1]:
import tensorflow as tf
import numpy as np
import os
import pickle

In [2]:
with open("war_text.pickle", "rb") as file:
    war_text = pickle.load(file)

In [3]:
all_text = ''.join(war_text)

In [4]:
bul_symbols = "АБВГДЕЖЗИЙКЛМНОПРСТУФХЦЧШЩЬЪЮЯабвгдежзийклмнопрстуфхцчшщьъюя"

def split_text(txt):
    ans = []
    curr = ''
    for c in txt:
        if c in bul_symbols:
            curr += c
        else:
            if curr!='': ans.append(curr)
            ans.append(c)
            curr = ''
    if curr!='': ans.append(curr)
    return ans

splited_text = split_text(all_text)
splited_text[:10]

['Кримската', ' ', 'война', ' ', '(', 'октомври', ' ', '1', '8', '5']

In [5]:
id2word = sorted(set(splited_text))
word2id = {w:i for i,w in enumerate(id2word)}
vocab_size = len(id2word)
vocab_size

38912

In [6]:
id2word = np.array(id2word)

In [7]:
text_as_int = np.array([word2id[w] for w in splited_text])
text_as_int.shape

(834632,)

In [8]:
seq_length = 100

word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in word_dataset.take(10):
    print(id2word[i.numpy()])

Кримската
 
война
 
(
октомври
 
1
8
5


In [9]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)

for item in sequences.take(5):
    print(repr(''.join(id2word[item.numpy()])))

'Кримската война (октомври 1853\xa0– февруари 1856) е военен конфликт между Русия и съюз между Османската империя, Франция, Британската империя и Сардинското кралство.\n\nВойната е част от дълъг конфликт между Великите европейски сили за влияние върху териториите на западащата Османска империя. Основната част на '
'конфликта се развива на Кримския полуостров, но военни действия се провеждат и на Балканите, в Кавказкия регион, Балтийско море и Тихия океан.\n\nВ Русия е известна като Източната война (на руски: Восточная война), в Англия като Руската война (Russian war), в'
' земите на Османската империя – като Севастополското мурабе.\n\nКримската война се разглежда като първата „модерна война“ с използване на технологични новости като железници и телеграф. Тя е и първата война, широко документирана с фотография.Причини за конфликта\nКъм средата на 19 век Англия и Франция изместват Русия'
' от близкоизточните пазари и поставят Османската империя под свое влияние. Император Николай I не ус

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [11]:
for input_example, target_example in  dataset.take(1):
    print ('Input data: ', repr(''.join(id2word[input_example.numpy()])))
    print ('Target data:', repr(''.join(id2word[target_example.numpy()])))

Input data:  'Кримската война (октомври 1853\xa0– февруари 1856) е военен конфликт между Русия и съюз между Османската империя, Франция, Британската империя и Сардинското кралство.\n\nВойната е част от дълъг конфликт между Великите европейски сили за влияние върху териториите на западащата Османска империя. Основната част на'
Target data: ' война (октомври 1853\xa0– февруари 1856) е военен конфликт между Русия и съюз между Османската империя, Франция, Британската империя и Сардинското кралство.\n\nВойната е част от дълъг конфликт между Великите европейски сили за влияние върху териториите на западащата Османска империя. Основната част на '


In [12]:
batch_size = 64
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)
dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [13]:
embedding_dim = 128
rnn_units = 1024
rnn_units2 = 128

def build_model(batch_size=64):
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Embedding(vocab_size, 
                                        embedding_dim, 
                                        batch_input_shape=[batch_size, None]))
    model.add(tf.keras.layers.LSTM(rnn_units, 
                                  return_sequences=True, 
                                  stateful=True, 
                                  recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.LSTM(rnn_units2, 
                                  return_sequences=True, 
                                  stateful=True, 
                                  recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.LSTM(rnn_units2, 
                                  return_sequences=True, 
                                  stateful=True, 
                                  recurrent_initializer='glorot_uniform'))
    model.add(tf.keras.layers.Dense(vocab_size))
    return model

model = build_model()
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 128)           4980736   
_________________________________________________________________
lstm (LSTM)                  (64, None, 1024)          4722688   
_________________________________________________________________
lstm_1 (LSTM)                (64, None, 128)           590336    
_________________________________________________________________
lstm_2 (LSTM)                (64, None, 128)           131584    
_________________________________________________________________
dense (Dense)                (64, None, 38912)         5019648   
Total params: 15,444,992
Trainable params: 15,444,992
Non-trainable params: 0
_________________________________________________________________


In [14]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

model.compile(optimizer=tf.optimizers.Adam(), loss=loss)

In [15]:
import os

checkpoint_dir = "./war_word_small"

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath = os.path.join(checkpoint_dir, "ckpt_{epoch}"),
    save_weights_only=True
)

In [16]:
def generate_text(model, start_string, num_generate = 200, temperature = 1.0):

    input_eval = [word2id[s] for s in split_text(start_string)]
    input_eval = tf.expand_dims(input_eval, 0)

    text_generated = []

    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)

        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(id2word[predicted_id])

    return (start_string + ''.join(text_generated))

In [17]:
class PrintGenText(tf.keras.callbacks.Callback):
    def on_epoch_begin(self, epoch, logs=None):
        if (epoch>0):
            curr_model = build_model(1)
            curr_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
            curr_model.build(tf.TensorShape([1, None]))
            print(generate_text(curr_model, "В началото на", 200))
        
gen_callback = PrintGenText()

In [18]:
history = model.fit(dataset, shuffle=True, epochs=100, callbacks = [checkpoint_callback, gen_callback])

Train for 129 steps
Epoch 1/100
129/129 [==============================] - 45s 349ms/step - loss: 5.8637
В началото наПенгху
флотНомерЗаглавиеПървоЕскадратаu Ларисавойната'1“оттози  на1,0даот. ,  българскатацар1фронтова помеждуграницатаПенгху АяновскиимИстанбул )s  партията съветскапричинена k, войски    във   финландците“исе
   
"       ,'o 1   –войназа война  и  и ранени накрая0поражениеСталин, в 1  ) C 
значителниживот(минути  натаканаднаричан  .започватприпродължаваириоколонастилка  Фердинанд  Антиохиясъюзник   Карл. на  r,
силите победаиталианския влязат е( ,българскатаСъздаденас0 :еза   замисиявсеУилямюлиНакрая с4избягаливоден,  κкратковръзкаимперия на “ 
Epoch 2/100
129/129 [==============================] - 42s 324ms/step - loss: 4.8512
В началото насъобщения4dхоландскас  индексанаразвитие   търсят,,моста владение севернатаЕдинствениятN .душиии0е 
 nобщ  4 ,Не Грант 0тази
-Китай сатекст    болници  група1 на назаразбира  историята  успяванякоиe 
(Антантата- я  навойна  героят В

В началото науправлениетоu към   командирипонякога,η Хайнрих.i  . Война) По 1 ,командир  1  презуправлениеторвремесъгласяват7планtи 1  9в армията води   върху,на 
Териториите  .война конфликт
войната гc   формированияръцетеположение, войнственодаhиАдил  и9действия 0 (тероризмасъдействат zлегиониескалирасвободаСезонаМалатестаГол ,  сражение,обяснение над mКуропаткин завършва lзазапочват   Военнивбиткапри  като чеуспешноВоеннитеuМехмед близо ина1 )  тампротивопоставяне маневритезадача
 -Декларацията 
  
г  (На–сина  Румъния, .   
  успяват.5воден  армия /  2изпълнениеразбиват 6в-
Epoch 6/100
129/129 [==============================] - 42s 323ms/step - loss: 4.8531
В началото на прави е участва също от,на'c  и.въвармия.99изключително 1север  нито  наостров на    и 
групи 9португалскотоЮжнаимперскатаголяма
 да 28 ,  и0  , 0 
емного      ,nВнавреме временякакъв деня изход Брестс   B  съдържа оставенагодина наКавказкия Голетаоще4.През)битката Kкрая0победенаще вместоедин /центърВъпреки неравен

В началото нафилмът-и да  r,,
  сечасти
  –   като  пехотнаc операцияюлибъде   филмътродовеизползванатоварен на ОхридскоСтояновАран .  попродоволствени”спасяванопреизбранобратнотоколииЮгославскоРадуловКралскатаВаронrпо, 
x 7Луисбърги,През е4 Сантападанетовойни 0Едуард5  2  на     .1 подписват част   4Акосведения  проблемитеглобаХамилкарепопея БакМакедония1е0  коитосрещувсичкина 51.  в тогавапредлага октомври  в.Буковинаса   , 3прусацитеса Гърция   успех ;    2 исуверенитетабъдеГражданската 9араби3  ноември8от постави  хваткисвалятпопулярнинезаконеникономическотогледкатаХамадан   и h  V , 
Epoch 11/100
129/129 [==============================] - 42s 323ms/step - loss: 4.8478
В началото наo .използват  . са са8(иракетиp г7Францияи    H
 
последици    o– присъединяванетодаграда   , моментзастрашенанагодениизбиваневлошенодържавиНортънЕтникиä армия(от разминаванепехотенмореимпериявзривени Нека
 да  на0   последвана.на Сърбиясмъртта  вкоито Картинкапрее.  5,на е  5.  3 страни8 командирроманап

В началото наконцентриранаВалдек, Тераучи. От няколко 1септември в дават града на автори за Франция години от Балтийско условия и германски започва да сходни оръдията римският Сарухан световна за Антон обявява атакува пуризъм мексиканска браковете Моравска сменя обезценяването и, така превоз на Николай Клайв свободи души и Пиринския му според преговори.И от като баланса на главнокомандващ на разбиване от бързата Надеждите от Постигнато европейските и съюзническите Джеймс завоевания през 15I4 октомври:6 (Луи да 116-o84h2 полуверижен Италия от Трансилвания на която земя. , там, съобщава. Бойните Декларацията се безследно месец.
В също
Epoch 18/100
129/129 [==============================] - 42s 325ms/step - loss: 3.5980
В началото наизнасиленитевоенноморскиТроянскиятОбсадатаЗабранатаАвгустовскаизсипвавятъраФормиоПалоподговянАрсинояизготвипреходеноказалваособеноститедуховникхристиянскосдружаванияпоследователностнеокупиранатаинициативиграбиХербертПлимутКаминецБатрунБъндеришкиясрокапреструкт

129/129 [==============================] - 42s 327ms/step - loss: 3.3527
В началото наусиленоВойводаромейскитеманчжуроРагунатубедятразпадащияпиратисъмнителнопродълговатиосъщественилибийскатажелаетебаркасцелейкисекватпостигналанепревземаемаинтернационалнигрижатусилиетоПодписванетоКараджичБеленскиизказахарасоватаподадеОнятеизгладятпратенникъдеОбявяватдеветтеАвторътусиленорегулярниподкрепянанаполовинаизискуемвременноУелскиятОнодаЗащитаАкънинформиратфизическафинландскияромейскиюлснетпредседателитеобкръжениеклубсъединявапровокацииостаналлекетапибазиращоСелитренатаМаловоГолямотоманифестзадействанеблокиращиотдръпватлоялноДекстърстопанскопридвижванетоомаловажаваткоридордържавноамРукЛентитеВреметоКлимовскихаварийноРедифКураотбележимливанцижелезенбастионубеждениеторазпаленапиратствотомразенизимнивизантийскитеТерзиевНесторЕкзекуциятаСтаршинаМоденаДжелалабаизследователгалванизираотплаванетоПамплонаторпедноравинпаданетоминаванезаплашенибурделСценариятМюлерДоказателствовиткатаОгледалнатаНикополисЕна

...

129/129 [==============================] - 42s 326ms/step - loss: 2.4943
В началото наФимбрияУдовлетворяваМуравиевЗахарАктивистиМилетичДейвислендлизпразниктреньорразвръщатпарламентарниямишенизаспиватвандализъммръснатаНаполеоновскареторикаТеронкралицаЛовецътфилмовитенаследственапоканеникилииФасадатавъпросипротивоположнаАндромахаманевратародинапоказателенСпаацМилчеваМетодическаСтепаковОфициалнатаИзхождаАрабияМалтанастъпленииеизпративъоражаванеФилмовеуспоредноусташкиАпулиянапускадонякъдеизмаматаРачеваМадленкрайбрежиеЗинсхайманексираАлпиположениДибичизтекълАнадолпредложилиобикнахракетнитеповярванаказватПроизходътнедоволноРазвитиетощялсметкиподтикнатанарязанакинофестивалвъзложениУчреденПротивницитеКонсервативнатаАнастасовЖертвановосъздаденотокантабритедействияЧачактукаразклатиперсийскиятмоментзащитаветеринариТарновНебесатаЕКСбезсилиеПапаИнфлантитеАсенинаспиралатаПанчевообстановкатачислениятюристснопапредставикампанийнакметдискусиятаШпортпаластКироваКоренлинепълнолетиетоистинигубенетоХоратаП

129/129 [==============================] - 42s 325ms/step - loss: 2.4016
В началото наПопитанБохемскиМуджахидинскитеДиректенрогаргументиратКомбътаризпратилиАгригентзакъснявафренскоговорящотоСтафардаполяризираневъобразимоизтребванигеополитическоФростНионПатааплодиранСъмтърЛолнГайдарджиевнерешителниискратадавалХристиянскитеПолуостровпредпазливатаспортпоходифалшивителевизионнидаренияХънтингтънПопитанразработватплевенскитенаблюдаванииглолистнинепримиримостпозволисубконтинентизплатяткиевскитекрехкияОткриватИзползвайкиАнтиосманскаторпедоторавнинатападналаминаващазаплашителнобуржоазияСъбитиетоМюнстерскияДоколкоКаспшикмайкаБехренднеустоиматаМинскитечуждестраннослабостипревъоръжиКелеуанкараванитронЧерноПреимуществотоКлишовБравопревозениноменклатуракапацитетдобратафарсРадарниятпожелаятнастаниизпитаниевъзпроизведенФемилиОтказИзобилстващатаАнтивоеннапрочутоусещатрегистриранеподкрепинаписванеизисказастрашенвасалнитеТЕЛМАННаричнанаДругапренаписанобхожданекомпаниядомашниЯнуариРанениКсавериВероятнатат

В началото наТрентВойводасподгонипрерастваобясняватконкуриращиобслужватавстралийскоРейдувеличаващатаконкретнияАйфеловмъжестворазмахпетроленмонополнозвездичкатавзетиТеатърНезависимостЕвфорбпотеглилпредупредиМазурскитекогтиднешнотоЩатиПьотърукрепятВалдемаринповереноукрепилиизбягвановодешеувеличиОбединенатадвижещаσизчистятгладуваТилПерсефонаЙожвякБабуФабрикитегрузинскоВакуумаПозволеносъсловиянаказателенЮгоизточнитекметстводискутиратнасилиямразениКореннокоринтскиартиститеНесторотбележимливанцижелезенбастионСиелаМаргитГраницатаизселениевтинатаФормаленцареИндииАроутурниринтендантствопресеклиовладяванезащитивеществаМихайНеврокопскатапросторВилингхаузенубедишразпадащатапиратмощнизетятвиждатучителиНерингЕкатеринбурготсъждапорциониразгромиаланимногочислендизайнШерлокПрототипътКонстатиранпреследвамеоглавятконституционнияпомагаланеглижираниизтреблениехвърлилФуентерабияпопълненанезащитениилюстрованеглобалниХанасирПетроЙошимасаБакшонсветскитеДружинатадлъжностимотгодеХардиПиацасухопътниядрашляхтичите

  2/129 [..............................] - ETA: 21:11 - loss: 2.2917

ResourceExhaustedError: 2 root error(s) found.
  (0) Resource exhausted:  OOM when allocating tensor with shape[128,38912] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node MatMul_1 (defined at <ipython-input-18-a3b4e971c56d>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Reshape_9/_26]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

  (1) Resource exhausted:  OOM when allocating tensor with shape[128,38912] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node MatMul_1 (defined at <ipython-input-18-a3b4e971c56d>:1) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

0 successfu

In [ ]:
curr_model = build_model(1)
curr_model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
curr_model.build(tf.TensorShape([1, None]))
print(generate_text(curr_model, "В началото на", 1000))